In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import os
# get latest version of spark
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,624 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,048 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

# Set the partitions to 4 or 8.
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [ ]:
# Read in housing data from S3 Bucket
from pyspark import SparkFiles
url = "https://oleslamburgerbucket.s3.us-west-2.amazonaws.com/realtor-data.zip.csv"
spark.sparkContext.addFile(url)
us_housing_df = spark.read.csv(SparkFiles.get("realtor-data.zip.csv"), sep=",", header=True)

In [ ]:
# examine the dataset
us_housing_df.show(20)

In [ ]:
#get a list of data types for columns
us_housing_df.printSchema()

In [ ]:
# Convert numeric values to floats, integers, and dates
us_housing_df = us_housing_df.withColumn('bed',us_housing_df['bed'].cast('float'))
us_housing_df = us_housing_df.withColumn('bath',us_housing_df['bath'].cast('Int'))
us_housing_df = us_housing_df.withColumn('acre_lot',us_housing_df['acre_lot'].cast('float'))
us_housing_df = us_housing_df.withColumn('zip_code',us_housing_df['zip_code'].cast('Int'))
us_housing_df = us_housing_df.withColumn('house_size',us_housing_df['house_size'].cast('float'))
us_housing_df = us_housing_df.withColumn('prev_sold_date',us_housing_df['prev_sold_date'].cast('date'))
us_housing_df = us_housing_df.withColumn('price',us_housing_df['price'].cast('float'))











In [ ]:
us_housing_df.count()

In [ ]:
df = us_housing_df.toPandas()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# visualizing the missing values by column
sns.barplot(df.isnull().sum().sort_values(ascending=False), palette='bright')
plt.ylim(0, 70000)
plt.xticks(rotation = 90);

Data Cleaning

In [ ]:
# drop the prev_sold_date column because it is missing half the values
df = df.drop('prev_sold_date', axis = 1)

In [ ]:
# drop null values for city, zip_code and price
df.dropna(subset=['city', 'zip_code', 'price'], inplace=True)

In [ ]:
# drop all rows where bed and bath values are missing
df = df.drop(df[(df['bed'].isnull()) & (df['bath'].isnull())].index, axis = 0)

In [ ]:
# drop all rows where acre_lot and house_size values are missing
df = df.drop(df[(df['acre_lot'].isnull()) & (df['house_size'].isnull())].index, axis = 0)

In [ ]:
# drop all rows where bath and house_size values are missing
df = df.drop(df[(df['bed'].isnull()) & (df['house_size'].isnull())].index, axis = 0)

In [ ]:
# drop all rows where bed and house_size values are missing
df = df.drop(df[(df['bath'].isnull()) & (df['house_size'].isnull())].index, axis = 0)

In [ ]:
# use linear regression to predict missing values for house_size
# the features we will use are bed, bath, acre_lot and price
df.corr()['house_size'].sort_values(ascending=False)[1:]

In [ ]:
# gather training data from df and drop nulls
house_size_df = df[['bed', 'bath', 'acre_lot', 'price', 'house_size']].dropna()

In [ ]:
# check for outliers that may skew results
sns.boxplot(house_size_df['house_size'].sort_values(ascending=False))
house_size_df['house_size'].sort_values(ascending=False)[:10]

In [ ]:
# drop rows with house_size values over 5000
house_size_df = house_size_df[(house_size_df['house_size'] < 5000) & (house_size_df['house_size'] > 400)]

sns.boxplot(house_size_df['house_size'].sort_values(ascending=False))

In [ ]:
len(house_size_df)

In [ ]:
house_size_df

In [ ]:
# separate features and target
# split the data
X = house_size_df.drop('house_size', axis = 1)
y = house_size_df['house_size']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
# scale the data
house_scaler = StandardScaler()
X_train_scaled = house_scaler.fit_transform(X_train)
X_test_scaled = house_scaler.transform(X_test)

In [ ]:
# create and train the model
model = LinearRegression()
model.fit(X_train_scaled,y_train)

In [ ]:
# get predictions and evaluate
y_hat = model.predict(X_test_scaled)
print(f'mae: {mean_absolute_error(y_test, y_hat)}')
print(f'mse: {mean_squared_error(y_test, y_hat)}')
print(f'mse: {np.sqrt(mean_squared_error(y_test, y_hat))}')

In [ ]:
# mean absolute error is 20% of the average house size, not the best accuracy but also not too bad.
420/house_size_df['house_size'].mean()

In [ ]:
# use a sequential model to make predictions instead
tf_model = Sequential()
tf_model.add(Dense(28, activation = 'relu'))
tf_model.add(Dense(14, activation = 'relu'))
tf_model.add(Dense(7, activation = 'relu'))
tf_model.add(Dense(1))

tf_model.compile(loss='mean_absolute_error', optimizer = 'adam', metrics = ['mae'])

In [ ]:
stop = EarlyStopping(patience = 1, monitor = 'val_loss')
tf_model.fit(X_train_scaled, y_train, epochs = 100, validation_data=(X_test_scaled, y_test), callbacks = [stop])

In [ ]:
# sequential model performed a little bit better
# gather the rows and features where house_size is null and store in df
X_house_size = df[df['house_size'].isnull()][['bed', 'bath', 'acre_lot', 'price']]

In [ ]:
# scale the data
X_house_size_scaled = house_scaler.fit_transform(X_house_size)

In [ ]:
# use the sequential model to predict
house_size_preds = tf_model.predict(X_house_size_scaled)

In [ ]:
# grab indices of rows where house_size is missing data
house_size_null_index = df[df['house_size'].isnull()].index

# reshape predictions to be 1 dimension and round the numbers off
# put into a series and set the index to match the index of our df of missing house_size values
house_size_preds_series = pd.Series(np.round(house_size_preds.reshape(-1))).set_axis(house_size_null_index)

# fill the values in place and update the house_size column
df['house_size'] = df['house_size'].fillna(house_size_preds_series)

In [ ]:
df.isnull().sum()

In [ ]:
# Now do the same for the acre_lot column using the house_size, price and state 'house_size', 'state', 'price', 'acre_lot'
acre_lot_df = df[['house_size', 'state', 'price', 'acre_lot']].dropna()

In [ ]:
# check for outliers that may skew results
sns.boxplot(acre_lot_df['acre_lot'].sort_values(ascending=False))
house_size_df['acre_lot'].sort_values(ascending=False)[:10]

In [ ]:
# drop rows with acre_lot values over 10 acres and less than 0.01
acre_lot_df = acre_lot_df[(acre_lot_df['acre_lot'] <= 10) & (acre_lot_df['acre_lot'] > 0.01)]
sns.boxplot(acre_lot_df['acre_lot'].sort_values(ascending=False))

In [ ]:
# get dummies for the state column
acre_lot_df = pd.get_dummies(acre_lot_df)

In [ ]:
# separate features and target
# split the data
X = acre_lot_df.drop('acre_lot', axis = 1)
y = acre_lot_df['acre_lot']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
# scale the data
acre_scaler = StandardScaler()
X_train_scaled = acre_scaler.fit_transform(X_train)
X_test_scaled = acre_scaler.transform(X_test)

In [ ]:
# create and train the model
model = LinearRegression()
model.fit(X_train_scaled,y_train)

In [ ]:
# get predictions and evaluate
y_hat = model.predict(X_test_scaled)
print(f'mae: {mean_absolute_error(y_test, y_hat)}')
print(f'mse: {mean_squared_error(y_test, y_hat)}')
print(f'mse: {np.sqrt(mean_squared_error(y_test, y_hat))}')

In [ ]:
# mae is even higher than the average acre_lot size so this model is very inaccurate with the available features.
acre_lot_df['acre_lot'].mean()

In [ ]:
# few of the properties have over 50 acres so we will drop anything over 50 acres
# then fill null values with the mean instead
df = df[df['acre_lot'] <= 50]

In [ ]:
df['acre_lot'].fillna(df['acre_lot'].mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
# drop remaining missing values
df = df.dropna(subset=['bed', 'bath'])

In [ ]:
df.isnull().sum()

In [ ]:
# clean up more outliers in the dataset
df = df[(df['house_size'] < 10000) & (df['house_size'] > 400)]
df = df[(df['acre_lot'] < 10) & (df['acre_lot'] > 0.01)]
df = df[df['bed'] <= 10]
df = df[df['bath'] <= 10]

In [ ]:
sns.boxplot(df['price'])

In [ ]:
df['price'].sort_values(ascending=False)[:10]

In [ ]:
len(df[df['price'] > 10000000])

In [ ]:
# remove price values over 10 million and under 25000
df = df[(df['price'] < 10000000) & (df['price'] > 25000)]

In [ ]:
# reset the index
df.reset_index(inplace=True, drop = True)

In [ ]:
df.head()

In [ ]:
df.to_csv('cleaned_realtor_data.csv', index = False)

Visualizations

In [ ]:
df.corr()['price'].sort_values(ascending=False)[1:]

In [ ]:
# create heatmap of correlations
sns.heatmap(df.corr(), annot = True, cmap='viridis')

In [ ]:
# scatterplot showing correlation between house_size and price
sns.scatterplot(data=df.head(100000), x='house_size', y='price', alpha = .5)

In [ ]:
# clear correlation between bath and price using a boxplot
sns.boxplot(data=df, x='bath', y = 'price', palette = 'bright', hue = 'bath')
plt.legend(loc = (1.1,0))

In [ ]:
sns.boxplot(data=df, x='bed', y = 'price', palette = 'bright', hue = 'bed')
plt.legend(loc = (1.1,0))

In [ ]:
sns.histplot(data=df, x = 'price', bins = 30)
plt.ylim(0,50000)

Prepare the data for making price predictions with Scikit-Learn and Tensorflow

In [ ]:
# drop the for_sale column because there is only 1 unique value
df['status'].unique()
df = df.drop('status', axis = 1 )

In [ ]:
df['city'].nunique()

In [ ]:
# Cities has nearly 5000 unique values so we will convert the cities into their respective value counts/frequencies
city_counts = df['city'].value_counts()
df['city'] = df['city'].apply(lambda x: city_counts.loc[x])

In [ ]:
df['zip_code'].nunique()

In [ ]:
# we will do the same with zip_code to reduce the size of the values
zip_code_counts = df['zip_code'].value_counts()
df['zip_code'] = df['zip_code'].apply(lambda x: zip_code_counts.loc[x])

In [ ]:
# use the OneHotEncoder to get dummies for the state column
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output= False).set_output(transform= 'pandas')

In [ ]:
ohetransform = ohe.fit_transform(df[['state']])

In [ ]:
df = pd.concat([df, ohetransform], axis = 1).drop(['state'], axis = 1)

In [ ]:
df.head()

In [ ]:
# split the data into features and the target value
X = df.drop('price', axis = 1)
y = df['price']

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.20, random_state=42)

In [ ]:
# scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
joblib.dump(scaler, 'scaler.pkl')

Begin Training The Model

In [ ]:
# create linear regression model
# fit to training data
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)

In [ ]:
# make predictions
lr_preds = linear_model.predict(X_test_scaled)

In [ ]:
# use metrics to analyze results
print(f'mae: {mean_absolute_error(y_test, lr_preds)}')
print(f'mse: {mean_squared_error(y_test, lr_preds)}')
print(f'rmse: {np.sqrt(mean_squared_error(y_test, lr_preds))}')

In [ ]:
# the linear regression model does not produce desireable results
# we will now try RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators = 130)

In [ ]:
rf_model.fit(X_train_scaled, y_train)

In [ ]:
rf_preds = rf_model.predict(X_test_scaled)
print(f'mae: {mean_absolute_error(y_test, rf_preds)}')
print(f'mse: {mean_squared_error(y_test, rf_preds)}')
print(f'rmse: {np.sqrt(mean_squared_error(y_test, rf_preds))}')

In [ ]:
# RandomForestRegressor is nearly perfect at predicting housing prices
sns.kdeplot(data=rf_preds, label = 'predictions')
sns.kdeplot(data=y_test, label = 'actual')
plt.legend()

In [ ]:
joblib.dump(rf_model, "housing-price-model.pkl")

Our RandomForestRegressor Has Nearly Perfect Accuracy

In [ ]:
city_series = df['city'].value_counts()

In [ ]:
zip_series = df['zip_code'].value_counts()

In [ ]:
# Save series to csv for preprocessing (converting data to their value counts) with new data
city_series.to_csv('city_series.csv')
zip_series.to_csv('zip_series.csv')

In [ ]:
# get dataframe where each row has a unique state
# this df will be used to create a small dataframe to allow for one hot encoding when new data is added
unique_states_df = df.drop_duplicates('state')
unique_states_df = unique_states_df.drop(['price', 'status'], axis = 1)
unique_states_df.head()

In [ ]:
unique_states_df.to_csv('unique_states_df.csv', index = False)

In [ ]:
# load in necessary files to run custom function below
zip_series = pd.read_csv('zip_series.csv')
city_series = pd.read_csv('city_series.csv')
loaded_scaler = joblib.load('housing_scaler.pkl')
loaded_model = joblib.load('housing_model.pkl')

In [ ]:
# create a function to automatically preprocess new data
def preprocessing(unique_states_df, user_row, city_series, zip_series):

    # convert state and city to title case
    user_row['city'] = user_row['city'].str.title()
    user_row['state'] = user_row['state'].str.title()

    # read in city and zip csv files
    city_series = city_series.copy(deep=True)
    zip_series = zip_series.copy(deep=True)

    # convert to a series
    city_series.set_index('Unnamed: 0', inplace=True)
    city_series = city_series['city']

    try:
        # convert to city count
        user_row['city'] = city_series.loc[user_row['city'].iloc[0]]

    except:
        # if city not in training data, replace with 1
        city = user_row['city'].iloc[0]
        print(f'Model has never seen "{city}" before. Estimate may be inaccurate.')
        user_row['city'] = 1

    # convert to a series
    zip_series.set_index('Unnamed: 0', inplace=True)
    zip_series = zip_series['zip_code']

    try:
        # convert to zip count
        user_row['zip_code'] = zip_series.loc[user_row['zip_code'].iloc[0]]

    except:
        # if zip not in training data, replace with 1
        zip = user_row['zip_code'].iloc[0]
        print(f'Model has never seen zipcode "{zip}" before. Estimate may be inaccurate.')
        user_row['zip_code'] = 1

    unique_states_df['city'] = unique_states_df['city'].apply(lambda x: int(x == 0))

    # add row to unique df
    new_df = pd.concat([unique_states_df, user_row])

    # create encoder
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output= False).set_output(transform= 'pandas')
    ohetransform = ohe.fit_transform(new_df[['state']])
    new_df = pd.concat([new_df, ohetransform], axis=1).drop(['state'], axis=1)

    new_df = loaded_scaler.transform(new_df)

    return [new_df[-1]]


In [ ]:
# run a test on data it hasn't seen before
target_value = df.loc[[X_test.index[0]]]['price'].iloc[0]
print(f'price target: ${target_value}')
test_row = df.loc[[X_test.index[0]]]
test_row = test_row.drop(['status', 'price'], axis = 1)
test_row

In [ ]:
test = preprocessing(unique_states_df, test_row, city_series, zip_series)

In [ ]:
loaded_model.predict(test)[0]

The RandomForestRegressor Was 100% Accurate With New Data That Was Preprocessed Using Our Function

Making Price Predictions with Tensorflow

In [ ]:
tf_price_model = Sequential()

tf_price_model.add(Dense(256, activation = 'relu', input_shape = (25,)))
tf_price_model.add(Dense(256, activation = 'relu'))
tf_price_model.add(Dense(128, activation = 'relu'))
tf_price_model.add(Dense(64, activation = 'relu'))
tf_price_model.add(Dense(1))

tf_price_model.compile(loss='mean_absolute_error', optimizer = 'adam', metrics = ['mae'])
tf_price_model.summary()

In [ ]:
stop = EarlyStopping(patience = 3, monitor = 'val_loss')
trained_tf_model = tf_price_model.fit(X_train_scaled, y_train, epochs = 300, validation_data=(X_test_scaled, y_test), callbacks=[stop])

In [ ]:
history = pd.DataFrame(tf_price_model.history.history)

In [ ]:
history.plot()

In [ ]:
tf_model_preds = tf_price_model.predict(X_test_scaled)

In [ ]:
# print metrics
print(f'mae: {mean_absolute_error(y_test, tf_model_preds)}')
print(f'mse: {mean_squared_error(y_test, tf_model_preds)}')
print(f'rmse: {np.sqrt(mean_squared_error(y_test, tf_model_preds))}')

The Sequential Model is Not Nearly as Accurate as the RandomForestRegressor and Took Several Hours to Run

The Recommended Model For This Dataset is a RandomForestRegressor With n_estimators = 130